In [23]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from iris import irisRequests
import ngl
import datetime
from scipy.signal import butter, lfilter, freqz

In [24]:
def butter_lowpass(cutoff, fs, order=4):
    nyquist = 0.5 * fs
    normal_cutoff = cutoff / nyquist
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return b, a

def butter_lowpass_filter(data, cutoff, fs, order=4):
    b, a = butter_lowpass(cutoff, fs, order=order)
    y = lfilter(b, a, data)
    return y

In [2]:
rootpath = "csv_24/"

In [3]:
regions = {}
regions['japan'] = (30, 45.5, 129.5, 146)
regions['italy'] = (35,46,6, 19)

In [4]:
station_list = ngl.ngl_process_list(ngl.ngl_24h_2w) # daily measurements, with 2 weeks delay

In [5]:
station_info = {}
for name, region in regions.items():
    station_names, station_lats, station_lons  = ngl.get_all_stations_box(station_list, *region)
    station_info[name]= (station_names, station_lats, station_lons)

In [7]:
station_data = {}
for name, s_info in station_info.items():
    for s_cnt, s_name in enumerate(s_info[0]):
        df, status = ngl.ngl_retrieve_24h(rootpath, s_name)
        print(s_name, status)
        station_data[s_name] = df

9071 loaded
AIRA loaded
CHOF loaded
DOKD loaded
G000 loaded
G001 loaded
G002 loaded
G003 loaded
G004 loaded
G006 loaded
G007 loaded
G008 loaded
G009 loaded
G010 loaded
G011 loaded
G012 loaded
G013 loaded
G014 loaded
G015 loaded
G016 loaded
G017 loaded
G018 loaded
G019 loaded
G020 loaded
G021 loaded
G022 loaded
G023 loaded
G024 loaded
G025 loaded
G026 loaded
G027 loaded
G028 loaded
G029 loaded
G030 loaded
G031 loaded
G032 loaded
G033 loaded
G035 loaded
G036 loaded
G037 loaded
G038 loaded
G039 loaded
G040 loaded
G041 loaded
G042 loaded
G043 loaded
G044 loaded
G045 loaded
G046 loaded
G047 loaded
G048 loaded
G049 loaded
G050 loaded
G051 loaded
G052 loaded
G053 loaded
G054 loaded
G055 loaded
G056 loaded
G058 loaded
G059 loaded
G060 loaded
G061 loaded
G062 loaded
G063 loaded
G064 loaded
G065 loaded
G066 loaded
G067 loaded
G068 loaded
G069 loaded
G070 loaded
G071 loaded
G073 loaded
G074 loaded
G075 loaded
G076 loaded
G077 loaded
G078 loaded
G079 loaded
G080 loaded
G081 loaded
G082 loaded
G083

In [78]:
start_time = datetime.datetime(2012, 1, 1, 0, 0, 0)
end_time =  datetime.datetime(2024, 1, 1, 0, 0, 0)
catalogs = {}
for name, region in regions.items():
    download_url =irisRequests.url_events_box(start_time, end_time, region[0], region[1], region[2], region[3], minmag=4, magtype="MW")
    df = pd.read_csv(download_url, sep="|", comment="#", parse_dates=["Time"])
    df.sort_values(by="Time", inplace=True)
    df.reset_index(inplace=True, drop=True)
    catalogs[name] = df